# Time Decay
**Truth Premise: When a day ends and a new day begins in life of options premium, its value depreciates because of time decay because it gets closer to expiry and the probability of its contract going as expected by the buyer gets lesser.**

## Strategy
1. Short both put and call at day closing right before the expiry
2. Square of in the morning

## Pseudocode
```python
nifty_at_0320
at_the_money_call_strike
at_the_money_put_strike
call_strike_premium_at_0320
put_strike_premium_at_0320
call_strike_premium_at_0915
put_strike_premium_at_0915
call_strike_premium_at_0916
put_strike_premium_at_0916
call_strike_premium_at_0917
put_strike_premium_at_0917
call_strike_premium_at_0918
put_strike_premium_at_0918
pnl_0915
pnl_0916
pnl_0917
pnl_0918
```

In [1]:
import datetime as dt
import utils as ut
import pandas as pd
import icharts as ic
from functools import cache
from constants import *


TEST_START = dt.datetime.strptime("2021-01-01", "%Y-%m-%d")
TEST_END = dt.datetime.strptime("2023-12-31", "%Y-%m-%d")
SYMBOL = "NIFTY 50"
IC_SYMBOL = "NIFTY"
INTERVAL = ut.INTERVAL_MIN1
EXCHANGE = ut.EXCHANGE_NSE
pickle_file_name = "ocdf_2024_02_17.pkl"
# pickle_file_name = "test_analyzer_ocdf_2024_02_17.pkl"

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", 200)
# pd.set_option('precision', 2)
pd.set_option("display.precision", 2)
# pd.set_option('display.float_format', lambda x: '%.2f' % x)

def build_date_range(date_start, date_end, symbol):
    date_range = []
    cur_date = date_start
    while cur_date < date_end:
        if cur_date.weekday() not in [5, 6]:
            has_data, _ = ut.has_data(symbol, cur_date, interval=INTERVAL, exchange=EXCHANGE)
            if has_data:
                date_range.append(cur_date)
        cur_date += dt.timedelta(days=1)
    return date_range

all_dates = pd.DataFrame({"trade_date": build_date_range(TEST_START, TEST_END, SYMBOL)})
all_dates_shuffled = all_dates.sample(frac=1, random_state=42)

train_size = int(0.5 * len(all_dates_shuffled))
train_dates = all_dates_shuffled.iloc[:train_size]
test_dates = all_dates_shuffled.iloc[train_size:]
train_dates = all_dates
train_dates = train_dates.sort_values(by="trade_date")
train_dates.set_index("trade_date", inplace=True)
test_dates = test_dates.sort_values(by="trade_date")
test_dates.set_index("trade_date", inplace=True)

def get_intraday_data(date):
    return ut.get_data(symbol=SYMBOL, date=date, interval=INTERVAL, exchange=EXCHANGE)

def get_symbol_first_candle(symbol, trade_date):
    data = ut.get_data(symbol=SYMBOL, date=trade_date, interval=INTERVAL, exchange=EXCHANGE)
    return data.iloc[0].open, data.iloc[0].high, data.iloc[0].low, data.iloc[0].close

def get_first_candle_close(symbol, trade_date):
    data = ut.get_data(symbol=SYMBOL, date=trade_date, interval=INTERVAL, exchange=EXCHANGE)
    return data.iloc[0].close

def get_last_trading_day(date):
    return ut.get_last_trading_day(SYMBOL, date, interval=INTERVAL, exchange=ut.EXCHANGE_NSE)

train_dates["expiry"] = pd.NA
train_dates["expiry"] = train_dates.apply(lambda row: ut.find_closest_expiry(SYMBOL, row.name), axis=1)

def get_nifty_price(trade, t):
    data = ut.get_data(symbol=SYMBOL, date=trade.name.date(), interval=INTERVAL, exchange=EXCHANGE)
    try:
        return data.loc[data.index.time == t].iloc[0].open
    except IndexError:
        return pd.NA
    except AttributeError as e:
        return pd.NA

train_dates = train_dates.copy()
train_dates.loc[:, "nifty_at_0918"] = train_dates.apply(lambda r: get_nifty_price(r, dt.time(hour=9, minute=15)), axis=1)
train_dates.loc[:, "nifty_at_1520"] = train_dates.apply(lambda r: get_nifty_price(r, dt.time(hour=15, minute=29)), axis=1)
train_dates.loc[:, "nifty_diff"] = train_dates.nifty_at_1520 - train_dates.nifty_at_0918
train_dates = train_dates.loc[train_dates.nifty_at_0918.notna()]
train_dates["atm_strike"] = train_dates.apply(lambda trade: round(trade.nifty_at_0918 / 50) * 50, axis=1)

In [2]:
def get_premium_df(trade, strike_price, td, option_type, tm):
    pr = ic.get_opt_pre_df(symbol=IC_SYMBOL, expiry=trade.expiry, cur_dt=td, strike_price=strike_price, option_type=option_type)
    if type(pr) == type(pd.NA) or pr.shape[0] == 0:
        return pd.NA
    x = pr.loc[(pr.index.date == td) & (pr.index.time >= tm)]
    # # print(x.iloc[0])
    if x.shape[0] == 0:
        # print(tm)
        # print(td)
        # print(pr.loc[(pr.index.date == td)])
        # print(trade)
        return pd.NA
    return x.iloc[0].open

# for i in range(-500, 750, 50):
#     train_dates["cur_put_atm_strike"] = train_dates["atm_strike"] + i
#     key1 = f"put_at_0918_{i}"
#     key2 = f"put_at_0320_{i}"
#     train_dates[key2] = train_dates.apply(lambda trade: get_premium_df(trade, trade.cur_put_atm_strike, trade.trade_day_before_expiry.date(), OPTION_TYPE_PUT, dt.time(hour=15, minute=20)), axis=1)
#     train_dates[key1] = train_dates.loc[train_dates[key2].notna()].apply(lambda trade: get_premium_df(trade, trade.cur_put_atm_strike, trade.trade_day_before_expiry.date() + dt.timedelta(days=1), OPTION_TYPE_PUT, dt.time(hour=9, minute=18)), axis=1)
#     pnl_key = f"put_pnl_0918_{i}"
#     train_dates[pnl_key] = train_dates.loc[train_dates[key2].notna()][key2] - train_dates.loc[train_dates[key2].notna()][key1]
#     print(f"Put PnL at 9:18 for strike: {i}, Total: {train_dates[pnl_key].sum()}, Per Day: {train_dates[pnl_key].mean()}, Days: {train_dates.loc[train_dates[pnl_key].notna()].shape[0]}")

"""
Results for call 0320 selling
Call PnL at 3:20 for strike: -500, Total: 339.15, Per Day: 0.9216032608695651, Days: 368
Call PnL at 3:20 for strike: -450, Total: 372.7, Per Day: 1.0127717391304347, Days: 368
Call PnL at 3:20 for strike: -400, Total: 399.15, Per Day: 1.0846467391304346, Days: 368
Call PnL at 3:20 for strike: -350, Total: 420.94999999999993, Per Day: 1.1438858695652172, Days: 368
Call PnL at 3:20 for strike: -300, Total: 428.20000000000005, Per Day: 1.1635869565217392, Days: 368
Call PnL at 3:20 for strike: -250, Total: 411.8000000000001, Per Day: 1.1190217391304351, Days: 368
Call PnL at 3:20 for strike: -200, Total: 339.0499999999999, Per Day: 0.9213315217391301, Days: 368
Call PnL at 3:20 for strike: -150, Total: 222.60000000000008, Per Day: 0.6048913043478263, Days: 368
Call PnL at 3:20 for strike: -100, Total: -35.999999999999886, Per Day: -0.0980926430517708, Days: 367
Call PnL at 3:20 for strike: -50, Total: -368.65000000000003, Per Day: -1.0017663043478262, Days: 368
Call PnL at 3:20 for strike: 0, Total: -783.8999999999999, Per Day: -2.1301630434782606, Days: 368
Call PnL at 3:20 for strike: 50, Total: -861.9000000000002, Per Day: -2.342119565217392, Days: 368
Call PnL at 3:20 for strike: 100, Total: -862.0999999999998, Per Day: -2.3426630434782605, Days: 368
Call PnL at 3:20 for strike: 150, Total: -757.15, Per Day: -2.0574728260869564, Days: 368
Call PnL at 3:20 for strike: 200, Total: -812.2500000000002, Per Day: -2.2072010869565224, Days: 368
Call PnL at 3:20 for strike: 250, Total: -974.8, Per Day: -2.648913043478261, Days: 368
Call PnL at 3:20 for strike: 300, Total: -1024.6000000000004, Per Day: -2.7842391304347838, Days: 368
Call PnL at 3:20 for strike: 350, Total: -1266.650000000001, Per Day: -3.451362397820166, Days: 367
Call PnL at 3:20 for strike: 400, Total: -844.0000000000003, Per Day: -2.3123287671232884, Days: 365
Call PnL at 3:20 for strike: 450, Total: -484.5000000000013, Per Day: -1.360955056179779, Days: 356
Call PnL at 3:20 for strike: 500, Total: 443.50000000000034, Per Day: 1.2707736389684823, Days: 349
Call PnL at 3:20 for strike: 550, Total: 96.94999999999868, Per Day: 0.27939481268011146, Days: 347
Call PnL at 3:20 for strike: 600, Total: 240.4499999999996, Per Day: 0.7264350453172193, Days: 331
Call PnL at 3:20 for strike: 650, Total: 676.5500000000011, Per Day: 2.1409810126582314, Days: 316
Call PnL at 3:20 for strike: 700, Total: 2099.0999999999967, Per Day: 7.470106761565825, Days: 281
"""
# for i in range(0, 50, 50):
#     # for i in range(-500, 750, 50):
#     train_dates["cur_call_atm_strike"] = train_dates["atm_strike"] - i
#     key1 = f"call_at_0918_{i}"
#     key2 = f"call_at_0320_{i}"
#     train_dates[key1] = train_dates.apply(lambda trade: get_premium_df(trade, trade.cur_call_atm_strike, trade.name.date(), OPTION_TYPE_CALL, dt.time(hour=9, minute=18)), axis=1)
#     train_dates[key2] = train_dates.loc[train_dates[key1].notna()].apply(lambda trade: get_premium_df(trade, trade.cur_call_atm_strike, trade.name.date(), OPTION_TYPE_CALL, dt.time(hour=15, minute=20)), axis=1)
#     pnl_key = f"call_pnl_0320_{i}"
#     train_dates[pnl_key] = train_dates.loc[train_dates[key1].notna()][key1] - train_dates.loc[train_dates[key1].notna()][key2]
#     print(f"Call PnL at 3:20 for strike: {i}, Total: {train_dates[pnl_key].sum()}, Per Day: {train_dates[pnl_key].mean()}, Days: {train_dates.loc[train_dates[pnl_key].notna()].shape[0]}")


# Try buying
for i in range(0, 50, 50):
    # for i in range(-500, 750, 50):
    train_dates["cur_put_atm_strike"] = train_dates["atm_strike"] + i
    key1 = f"put_at_0918_{i}"
    key2 = f"put_at_0320_{i}"
    train_dates[key1] = train_dates.apply(lambda trade: get_premium_df(trade, trade.cur_put_atm_strike, trade.name.date(), OPTION_TYPE_PUT, dt.time(hour=9, minute=15)), axis=1)
    train_dates[key2] = train_dates.loc[train_dates[key1].notna()].apply(lambda trade: get_premium_df(trade, trade.cur_put_atm_strike, trade.name.date(), OPTION_TYPE_PUT, dt.time(hour=15, minute=29)), axis=1)
    pnl_key = f"put_pnl_0320_{i}"
    train_dates[pnl_key] = train_dates.loc[train_dates[key1].notna()][key2] - train_dates.loc[train_dates[key1].notna()][key1]
    print(f"Put PnL at 3:20 for strike: {i}, Total: {train_dates[pnl_key].sum()}, Per Day: {train_dates[pnl_key].mean()}, Days: {train_dates.loc[train_dates[pnl_key].notna()].shape[0]}")


Put PnL at 3:20 for strike: 0, Total: -2232.7000000000007, Per Day: -3.0294436906377213, Days: 737


In [3]:
pd.set_option("display.max_rows", 2000)
"""
Nifty at 0918 - 1520
-1323.2500000000164
-1.7930216802168244
739
"""
print(train_dates.nifty_diff.sum())
print(train_dates.nifty_diff.mean())
print(train_dates.shape[0])

-8097.499999999969
-10.97222222222218
739


In [4]:
train_dates

,expiry,nifty_at_0918,nifty_at_1520,nifty_diff,atm_strike,cur_put_atm_strike,put_at_0918_0,put_at_0320_0,put_pnl_0320_0
trade_date,,,,,,,,,
2021-01-01,2021-01-07,13996.1,14015.0,18.9,14000,14000,140.00,91.9,-48.1
2021-01-04,2021-01-07,14104.35,14142.45,38.1,14100,14100,93.30,66.3,-27.0
2021-01-05,2021-01-07,14075.15,14198.65,123.5,14100,14100,91.30,44.0,-47.3
2021-01-06,2021-01-07,14240.95,14127.7,-113.25,14250,14250,94.85,123.8,28.95
2021-01-07,2021-01-07,14253.75,14138.1,-115.65,14250,14250,65.15,113.0,47.85
2021-01-08,2021-01-14,14258.4,14347.95,89.55,14250,14250,119.70,74.8,-44.9
2021-01-11,2021-01-14,14474.05,14486.55,12.5,14450,14450,100.00,90.5,-9.5
2021-01-12,2021-01-14,14473.8,14570.55,96.75,14450,14450,100.60,36.0,-64.6
2021-01-13,2021-01-14,14639.8,14556.0,-83.8,14650,14650,91.00,100.3,9.3
